In [1]:
%load_ext autoreload
%autoreload 2
import moldb as mdb
from pony.orm import *

In [2]:
import requests
from pathlib import Path
if not Path('cubic_perovskites.db').exists():
    session = requests.get('https://cmr.fysik.dtu.dk/_downloads/03d2580a2f33d61c6998b803d2d72af0/cubic_perovskites.db')
    with open('cubic_perovskites.db', 'wb') as f:
        f.write(session.content)

In [3]:
db = mdb.Database()
entity = db.def_entity('Main', {
    'heat_of_formation_all': mdb.Required(float),
    'standard_energy': mdb.Required(float),
    'combination': mdb.Required(str)
})
db.bind(provider='sqlite', filename='/workspaces/molcrafts/moldb/example/cubic_perovskites.sqlite', create_db=True)
db.generate_mapping(create_tables=True)
db.load_ase('cubic_perovskites.db', entity, selection='combination')

In [22]:
db.show_table(entity, display_column=['id', 'heat_of_formation_all', ])

          Main table          
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ id ┃ heat_of_formation_all ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1  │ 1.2                   │
│ 2  │ 0.6                   │
│ 3  │ 0.54                  │
│ 4  │ 0.88                  │
│ 5  │ 1.78                  │
└────┴───────────────────────┘